In [1]:
import os
import openai
from langchain_openai.chat_models import ChatOpenAI
from dotenv import load_dotenv, find_dotenv


In [2]:
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
FIREWORKS_API_KEY = os.getenv("FIREWORKS_API_KEY")
Groq_API_KEY = os.getenv("GROQ_API_KEY")